In [1]:
import pandas as pd

import extract_coverage
import report_util
import tables


def create_rows(data, time, subject):
    data = data[data['time'] == time]
    data = data[data['subject'] == subject]
    fuzzers = sorted(data['fuzzer'].unique())
    rows = []
    values1 = data[data['fuzzer'] == 'Zeugma-Linked']['covered_branches']
    significance_level = 0.05 / (len(fuzzers) * (len(fuzzers) - 1) / 2)
    for fuzzer in fuzzers:
        values2 = data[data['fuzzer'] == fuzzer]['covered_branches']
        sig = 'color: red;' if report_util.mann_whitney(values1, values2) < significance_level else ''
        rows.append([time, subject, fuzzer, values2.median(), sig])
    return pd.DataFrame(rows, columns=['time', 'subject', 'fuzzer', 'median', 'sig'])


def create_sig_table(data, times):
    return pd.concat(
        [create_rows(data, time, subject) for time in times for subject in sorted(data['subject'].unique())])


times = [pd.to_timedelta(5, 'm'), pd.to_timedelta(3, 'h')]
coverage = extract_coverage.read_coverage_csv('../data/coverage.csv')
table = create_sig_table(coverage, times)
medians = tables.pivot(table, 'subject', 'fuzzer', 'median')
sigs = tables.pivot(table, 'subject', 'fuzzer', 'sig')
styler = tables.style_table(medians, precision=1, axis=0).apply(lambda _: sigs, axis=None)
display(styler)
print(styler.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
Subject & \multicolumn{2}{c}{Ant} & \multicolumn{2}{c}{Bcel} & \multicolumn{2}{c}{Closure} & \multicolumn{2}{c}{Maven} & \multicolumn{2}{c}{Nashorn} & \multicolumn{2}{c}{Rhino} & \multicolumn{2}{c}{Tomcat} \\
 & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H \\
Fuzzer &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
BeDivFuzz-Simple & \color{red} 755.0 & \color{red} 899.0 & \color{red} 1435.5 & \color{red} 1846.5 & \color{red} 9328.0 & \color{red} 11863.5 & \color{red} 590.5 & \color{red} 738.0 & \color{red} 3008.5 & \color{red} 3319.5 & \color{red} 2952.0 & \color{red} 3235.5 & \color{red} 274.5 & 341.0 \\
BeDivFuzz-Structure & \color{red} 786.5 & \color{red} 896.5 & \color{red} 1412.5 & \color{red} 1876.5 & \color{red} 9336.5 & \color{red} 11904.5 & \color{red} 578.5 & \color{red} 641.5 & \color{red} 2993.5 & \color{red} 3092.0 & \color{red} 2915.5 & \color{red} 3237.0 & \color{red} 161.0 & \color{red} 242.5 \\
RLChe